![NYC Skyline](nyc.jpg)

Welcome to New York City, one of the most-visited cities in the world. There are many Airbnb listings in New York City to meet the high demand for temporary lodging for travelers, which can be anywhere between a few nights to many months. In this project, we will take a closer look at the New York Airbnb market by combining data from multiple file types like `.csv`, `.tsv`, and `.xlsx`.

Recall that **CSV**, **TSV**, and **Excel** files are three common formats for storing data. 
Three files containing data on 2019 Airbnb listings are available to you:

**data/airbnb_price.csv**
This is a CSV file containing data on Airbnb listing prices and locations.
- **`listing_id`**: unique identifier of listing
- **`price`**: nightly listing price in USD
- **`nbhood_full`**: name of borough and neighborhood where listing is located

**data/airbnb_room_type.xlsx**
This is an Excel file containing data on Airbnb listing descriptions and room types.
- **`listing_id`**: unique identifier of listing
- **`description`**: listing description
- **`room_type`**: Airbnb has three types of rooms: shared rooms, private rooms, and entire homes/apartments

**data/airbnb_last_review.tsv**
This is a TSV file containing data on Airbnb host names and review dates.
- **`listing_id`**: unique identifier of listing
- **`host_name`**: name of listing host
- **`last_review`**: date when the listing was last reviewed

In [208]:
# Import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Begin coding here ...
# Use as many cells as you like

### Load data

In [209]:
df_price = pd.read_csv('data/airbnb_price.csv')
df_room = pd.read_excel('data/airbnb_room_type.xlsx')
df_review = pd.read_csv('data/airbnb_last_review.tsv', sep='\t')

print(df_price.head())
print("\n")
print(df_room.head())
print("\n")
print(df_review.head())

   listing_id        price                nbhood_full
0        2595  225 dollars         Manhattan, Midtown
1        3831   89 dollars     Brooklyn, Clinton Hill
2        5099  200 dollars     Manhattan, Murray Hill
3        5178   79 dollars  Manhattan, Hell's Kitchen
4        5238  150 dollars       Manhattan, Chinatown


   listing_id                                description        room_type
0        2595                      Skylit Midtown Castle  Entire home/apt
1        3831            Cozy Entire Floor of Brownstone  Entire home/apt
2        5099  Large Cozy 1 BR Apartment In Midtown East  Entire home/apt
3        5178            Large Furnished Room Near B'way     private room
4        5238         Cute & Cozy Lower East Side 1 bdrm  Entire home/apt


   listing_id    host_name   last_review
0        2595     Jennifer   May 21 2019
1        3831  LisaRoxanne  July 05 2019
2        5099        Chris  June 22 2019
3        5178     Shunichi  June 24 2019
4        5238          

### Merge data

In [210]:
airbnb_df = df_price.merge(df_room, on = 'listing_id') \
            .merge(df_review, on = 'listing_id')

airbnb_df.head()

listing_id        price  ...    host_name   last_review
0        2595  225 dollars  ...     Jennifer   May 21 2019
1        3831   89 dollars  ...  LisaRoxanne  July 05 2019
2        5099  200 dollars  ...        Chris  June 22 2019
3        5178   79 dollars  ...     Shunichi  June 24 2019
4        5238  150 dollars  ...          Ben  June 09 2019

[5 rows x 7 columns]

### Checks the type's columns

In [211]:
print(airbnb_df.info())
print("\n")
print(airbnb_df.shape)
print("\n")
print(airbnb_df.describe())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25209 entries, 0 to 25208
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   listing_id   25209 non-null  int64 
 1   price        25209 non-null  object
 2   nbhood_full  25209 non-null  object
 3   description  25199 non-null  object
 4   room_type    25209 non-null  object
 5   host_name    25201 non-null  object
 6   last_review  25209 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.5+ MB
None


(25209, 7)


         listing_id
count  2.520900e+04
mean   2.068922e+07
std    1.102928e+07
min    2.595000e+03
25%    1.202273e+07
50%    2.234391e+07
75%    3.037669e+07
max    3.645581e+07


In [212]:
airbnb_df.isna().sum()

listing_id      0
price           0
nbhood_full     0
description    10
room_type       0
host_name       8
last_review     0
dtype: int64

### Convert data types & remove missing values

In [213]:
airbnb_df.columns

Index(['listing_id', 'price', 'nbhood_full', 'description', 'room_type',
       'host_name', 'last_review'],
      dtype='object')

Set the columns type
- 'listing_id' = object
- 'price' = int
- 'nbhood_full' = object
- 'description' = object
- 'room_type' = object
- 'host_name' = object
- 'last_review' = date

In [214]:
airbnb_df['listing_id'] = airbnb_df['listing_id'].astype('object')
airbnb_df['price'] = airbnb_df['price'].str.replace(' dollars', '').astype('int')
airbnb_df['last_review'] = pd.to_datetime(airbnb_df['last_review'], format='%B %d %Y')

### Exploration Data Analyst

 1. What are the dates of the earliest and most recent reviews? 

In [215]:
# What are the dates of the earliest and most recent reviews?
first_reviewed = airbnb_df['last_review'].min()
last_reviewed = airbnb_df['last_review'].max()

# Convert datetime to date format
first_reviewed = first_reviewed.date()
last_reviewed = last_reviewed.date()

print('the earliest reviews is',first_reviewed)
print('the most recent reviews is',last_reviewed)

the earliest reviews is 2019-01-01
the most recent reviews is 2019-07-09


 2. How many of the listings are private rooms?

In [216]:
airbnb_df['room_type'].value_counts()

Entire home/apt    8458
Private room       7241
entire home/apt    2665
private room       2248
ENTIRE HOME/APT    2143
PRIVATE ROOM       1867
Shared room         380
shared room         110
SHARED ROOM          97
Name: room_type, dtype: int64

In [217]:
# Since there are differences in capitalization, make capitalization consistent
airbnb_df['room_type'] = airbnb_df['room_type'].str.lower()
private_room_count = airbnb_df[airbnb_df['room_type'] == 'private room'].shape[0]

print('nb_private_rooms:', private_room_count)

nb_private_rooms: 11356


 3. What is the average listing price?

In [218]:
avg_price = airbnb_df['price'].mean()

print(f'The average listing price: {avg_price:.2f}')

The average listing price: 141.78


 4. Combine the new variables into one DataFrame called review_dates with four columns in the following order: first_reviewed, last_reviewed, nb_private_rooms, and avg_price. 

In [219]:
review_dates = pd.DataFrame({
    'first_reviewed': [first_reviewed],
    'last_reviewed' : [last_reviewed],
    'nb_private_rooms' : [private_room_count],
    'avg_price' : [round(avg_price, 2)]
})

print(review_dates)

  first_reviewed last_reviewed  nb_private_rooms  avg_price
0     2019-01-01    2019-07-09             11356     141.78
